In [59]:
import numpy as np
from tabulate import tabulate
import pandas as pd
import csv
import ocel as ol
from sklearn.calibration import LabelEncoder


## preparing the cel
def prepare_ocel(fn):
    ocel = pd.read_csv('newdata/%s.csv' % fn)
    ocel['ocel:timestamp']=  pd.to_datetime(ocel['ocel:timestamp'])
    col_transform = {"ocel:eid": "Event_ID", "ocel:timestamp": "Timestamp", "ocel:activity": "Activity", "ocel:type:items": "Items",
                    "ocel:type:products": "Products", "ocel:type:customers":"Customers", "ocel:type:orders":"Orders",
                    "ocel:type:packages":"Packages"}
    ocel = ocel.rename(columns=col_transform).reindex(columns=[ 'Event_ID', 'Activity','Timestamp', 'weight', 'price', 'Items','Products', 'Customers', 'Orders', 'Packages'])
    ocel['Items']=  ocel['Items'].apply(lambda x: x.strip('][').split(', ')).apply(lambda x: [t.replace("'","") for t in x])
    ocel['Products']=  ocel['Products'].apply(lambda x: x.strip('][').split(', ')).apply(lambda x: [t.replace("'","") for t in x])
    ocel['Customers']=  ocel['Customers'].apply(lambda x: x.strip('][').split(', ')).apply(lambda x: [t.replace("'","") for t in x])
    ocel['Orders']=  ocel['Orders'].apply(lambda x: x.strip('][').split(', ')).apply(lambda x: [t.replace("'","") for t in x])
    ocel['Packages']=  ocel['Packages'].fillna('')
    ocel['Packages']=  ocel['Packages'].apply(lambda x: x.strip('][').split(', ')).apply(lambda x: [t.replace("'","") for t in x]).apply(lambda x: x[-1])
    ocel['Customers']=  ocel['Customers'].apply(lambda x: x[-1])
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(ocel['Activity'])
    act_dict = dict(zip(ocel['Activity'], encoded))
    encoded = encoder.fit_transform(ocel['Customers'])
    cust_dict = dict(zip(ocel['Customers'], encoded))
    ocel['Activity'] = ocel['Activity'].map(act_dict)
    ocel['Customers'] = ocel['Customers'].map(cust_dict)
    return ocel, act_dict, cust_dict
## flattening on order
def flatten(OCEL, on, printen = False):
    ocel_flat = OCEL.explode(on).reset_index(drop=True)
    if printen:
        print(f'size of normal OCEL:{OCEL.size}')
        display(OCEL[OCEL['Event_ID']== 594])
        print(f'size of flattend OCEL:{ocel_orders.size}')
        display(ocel_orders[ocel_orders['Event_ID']== 594])
        display(ocel_orders.head())
    return ocel_flat

#generating the csv files for the imput
drops_col_order = ["weight","price","Event_ID",'Products']
def gen_flatted_comp_csv(OCEL,flattedby, drops_col, printen = False):
    enriched_orders = OCEL.set_index(flattedby).drop(columns=drops_col)
    enriched_orders['amount_of_items'] = [len(t) for t in enriched_orders['Items']]
    enriched_orders.to_csv('./newdata/orders_complete_enriched.csv')
    
    single_orders = OCEL.set_index(flattedby)[['Activity','Timestamp']]
    single_orders.to_csv('./newdata/orders_complete_single.csv')
    if printen:
        display(enriched_orders[120:140])
        display(single_orders[120:140])

In [60]:
filename = "running-example"
ocel, act_dict, cust_dict = prepare_ocel(filename)
col = ["Items","Products","Orders",]
# for c in col:
#     print(c)
#     print(np.unique([len(t) for t in ocel[c]]))
print(act_dict)
print(cust_dict)
# display(ocel.head())
ocel_orders = flatten(ocel,'Orders')
gen_flatted_comp_csv(ocel_orders,'Orders',drops_col_order)

{'place order': 8, 'pick item': 7, 'confirm order': 0, 'item out of stock': 3, 'reorder item': 9, 'pay order': 5, 'create package': 1, 'send package': 10, 'failed delivery': 2, 'package delivered': 4, 'payment reminder': 6}
{'Marco Pegoraro': 12, 'Gyunam Park': 4, 'Majid Rafiei': 11, 'Junxiong Gao': 5, 'Seran Uysal': 14, 'Christina Rensinghof': 1, 'Wil van der Aalst': 16, 'Christine Dobbert': 2, 'Luis Santos': 8, 'Kefang Ding': 6, 'Mohammadreza Fani Sani': 13, 'Tobias Brockhoff': 15, 'Anahita Farhang Ghahfarokhi': 0, 'Mahnaz Qafari': 9, 'Claudia Graf': 3, 'Mahsa Bafrani': 10, 'Lisa Mannel': 7}


In [25]:
#### comparing if product and item match but is to hard to handle them

t1 = [len(t) for t in ocel['Items']]
t2 = [len(t) for t in ocel['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(ocel.size)
print(sum(t3))
print(sum(t3)/ocel.size)
print(len(t3) - sum(t3))
# display(ocel[t3])
# display(ocel[t3].explode('Products').groupby(['Products']).size().sort_values())
oceltemp = ocel[t3].copy()
t1 = [len(t) for t in oceltemp['Items']]
t2 = [len(t) for t in oceltemp['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(sum(t3))
df_agg = ocel.copy()
df_agg.Items = df_agg.Items.apply(len)
df_agg.Orders = df_agg.Orders.apply(len)
df_agg.Products = df_agg.Products.apply(len)
df_agg = df_agg.groupby('Packages').sum(numeric_only=True)
display(df_agg)

223670
4174
0.01866142084320651
18193
4174


,Event_ID,Activity,weight,price,Items,Products,Customers,Orders
Packages,,,,,,,,
,200453766.0,106165,23193.639,21642829.56,38114,35519,142937,18001
660001,567.0,17,28.844,32679.80,64,44,16,20
660002,527.0,15,12.819,11066.91,15,15,36,6
660003,1017.0,17,29.604,12595.80,48,32,20,16
660004,701.0,15,14.265,13979.88,33,27,33,12
...,...,...,...,...,...,...,...,...
661321,67062.0,15,0.840,269.97,3,3,21,3
661322,67071.0,15,1.485,387.00,3,3,45,3
661323,67081.0,15,1.449,239.97,3,3,30,3
